# Web Scraping Tutorial

This tutorial will teach you how to use Python to scrap and extract data from a web page. We will use two packages, `requests` to scrap the webpage and `BeautifulSoup` to extract the data.

Many good references on web scraping are available online. I would recommend the following resources:
1. Automate Boring Stuff with Python by Al Sweigart (2020) has a chapter on Web Scraping tutorial, which can be read [online](https://automatetheboringstuff.com/2e/chapter12/).
2. Web Scraping With Python by Ryan Mitchell (2018) is a bit old book but provides a comprehensive guide to the topic.

## Step 0: Getting to know the web page

In this tutorial, we will try to extract the cryptocurrency market prices from the CoinGecko website https://www.coingecko.com/.

Your first step should always be to familiarize yourself with the website you want to scrape. Take a look at the website and try to inspect the HTML elements on the webpage.

## Step 1: Scrap a web page

Now, we are ready to scrap a webpage we want to get the data from with the `requests` package. We will use the following functions:

* `requests.get('URL')` - make a request to the specified URL
* `r.status_code` - get the status code of the request
* `r.content` - get the binary content of the page

More functions in the `requests` package are available in [its documentation](https://requests.readthedocs.io/en/latest/).

In [45]:
# First, we will import the requests package
import requests

from os.path import join, exists, isfile, isdir, abspath, dirname, basename, realpath
from os import makedirs, listdir, pardir, getcwd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [46]:
parent_dir = abspath(join(join(getcwd(), pardir), pardir))
data_dir = join(parent_dir, 'data')
html_file = join(data_dir, 'coin_gecko.html')

In [4]:
# Request the webpage
url = "https://www.coingecko.com"
res = requests.get(url=url)

<Response [200]>

In [8]:
# Type of the request we've got
type(res)

requests.models.Response

In [29]:
# Check the status code
res.status_code == requests.codes.ok

True

In [30]:
# Get the header of the web page
res.headers['content-type']

'text/html; charset=utf-8'

In [12]:
# Get the content of the web page
res.content[:100]

b'<!DOCTYPE html>\n<html lang="en">\n<head>\n<script src="/cdn-cgi/apps/head/gYtXOyllgyP3-Z2iKTP8rRWGBm4.'

In [47]:
# Save the content of web page to the local drive
with open(html_file, "w") as f:
    f.write(res.text)

1329426

## Step 2: Extract data from the web page

After we crawled the web page and download it to the local disk, we will use `BeautifulSoup` package to parse HTML file and access the content. We will use the following functions:

**1. Load the web page to BeautifulSoup**
* `soup = BeautifulSoup(html_doc, 'html.parser')` - parse the HTML content to BeautifulSoup object

**2. Get the content of the element**
* `soup.title` - get the title of the page
* `soup.title.string` - get the string in the title tag
* `soup.h1` - get the H1 element in the web page
* `soup.h1.attrs` - get all attributes in the H1 element
* `soup.h1['class']` - get the class attribute in the H1 element

**3. Look for the element in the web page**
* `soup.find('HTML_tag')` - get the element from an HTML tag
* `soup.find_all('HTML_tag')` - get the list of elelemts that has the specified HTML tag
* `soup.select('CSS_selector')` - get the list of elements with the specified [CSS selector](https://www.w3schools.com/cssref/css_selectors.asp)

In [14]:
# First, we will import the BeautifulSoup from bs4 package
from bs4 import BeautifulSoup

In [48]:
# Load the web page and parse it to BeautifulSoup
with open(html_file, "r", encoding="utf-8") as f:
    soup = BeautifulSoup(f, "html.parser")

In [16]:
# Check the type of our soup object
type(soup)

bs4.BeautifulSoup

In [24]:
# Get all text in the web page
soup.get_text().strip().replace("\n", "")[:500]

'Cryptocurrency Prices, Charts, and Crypto Market Cap | CoinGeckoThis website uses cookies for functionality, analytics and advertising purposes as described in our Privacy Policy. If you agree to our use of cookies, please continue to use our site.OKContinue in appTrack prices in real-timeOpen AppContinue in appTrack prices in real-timeOpen AppEN LanguageEnglishDeutschEspañolFrançaisItalianojęzyk polskiLimba românăMagyar nyelvNederlandsPortuguêsSvenskaTiếng việtTürkçeРусский日本語简体中文繁體中文한국어العربي'

In [20]:
# Get the title of the page
soup.title

<title>Cryptocurrency Prices, Charts, and Crypto Market Cap | CoinGecko</title>

In [27]:
soup.h1.text.strip()

'Cryptocurrency Prices by Market Cap'

In [25]:
# We can also get the page title using soup.find() function
soup.find("title")

<title>Cryptocurrency Prices, Charts, and Crypto Market Cap | CoinGecko</title>

In [35]:
# Other HTML tags also work too
for i in range(1, 6):
    print(f"Total {len(soup.find_all(f'h{i}'))} H{i} tags")

Total 1 H1 tags
Total 4 H2 tags
Total 3 H3 tags
Total 0 H4 tags
Total 2 H5 tags


Now, we will extract the cryptocurrencies market price from the table.

In [37]:
# Get the table element in the web page
table = soup.find("table", attrs={"class": "table"})
type(table)

bs4.element.Tag

In [44]:
# Get the table headers
[x.text.strip() for x in table.thead.find_all("th")]

['',
 '#',
 'Coin',
 'Price',
 '1h',
 '24h',
 '7d',
 '24h Volume',
 'Mkt Cap',
 'FDV',
 'Last 7 Days']

In [53]:
# If there are > 1 elements that match the tagged, 
# use soup.find_all() to retrieve all of them as a list.
rows = [x for x in table.find_all("tr")]
cols = [x.find_all("td") for x in rows]
data = [[x.text.strip() for x in y] for y in cols]
len(rows)
len(cols)
len(data)
data[:5]

101

101

101

[[],
 ['',
  '1',
  'Bitcoin\n\n\nBTC',
  '$19,914.09',
  '0.0%',
  '3.7%',
  '3.6%',
  '$27,068,124,639',
  '$382,219,128,162',
  '$418,733,988,129',
  ''],
 ['',
  '2',
  'Ethereum\n\n\nETH',
  '$1,347.54',
  '0.1%',
  '4.3%',
  '0.9%',
  '$8,846,089,254',
  '$162,810,730,982',
  '?',
  ''],
 ['',
  '3',
  'Tether\n\n\nUSDT',
  '$0.999061',
  '-0.0%',
  '-0.1%',
  '-0.2%',
  '$31,670,303,985',
  '$67,885,610,166',
  '?',
  ''],
 ['',
  '4',
  'BNB\n\n\nBNB',
  '$290.49',
  '0.3%',
  '1.8%',
  '5.4%',
  '$570,182,494',
  '$47,450,255,431',
  '$47,984,919,219',
  '']]

In [ ]:
# Iterate over rows and get the data for each coin


## Step 3: Create data table and save as CSV file

Let's wrap our data table as the pandas's DataFrame and save it as a CSV file.

In [ ]:
import pandas as pd